# A. parse 一個 .html 檔中原po對下方推文的pair

In [3]:
from bs4 import BeautifulSoup

In [2]:
def find_list_of_comment_recomment_pairs(html_file):
    with open(html_file, "r") as f:
        html = f.read()
        
    pushs = html.split('<div class="push">')
    
    results = list()
    # [
    #     {"user":
    #      "type":
    #      "content":
    #      "reply":
    #      }
    # ]
    
    for push in pushs[1:]:
    
#         print(push)

        bs = BeautifulSoup(push)

        push_type = bs.find("span", class_="push-tag").text.strip()
        push_user = bs.find("span", class_="push-userid").text.strip()
        push_content = bs.find("span", class_="push-content").text.strip().lstrip(": ")

        
        # 若是該推文版主有回...
        if not push.endswith("</div>") and not "</script>" in push:
            
            # 先抓出版主回的內容
            reply = push.split("</div>")[-1].split("<span")[0].strip()
            if len(reply) == 0:
                reply = None
            
            
            # 如果目前results是空的 (初始條件)
            if len(results) == 0:
                results.append({"user": push_user, "type": push_type, "content": push_content, "reply": reply})
            
            # 如果目前results不是空的，那麼就要檢查這一筆推文的作者，是不是跟results最後一筆的作者相同，如果相同的話要merge
            else:
                
                # 若該推文的作者與results最後一筆紀錄的作者相同的話，把內容合併在一起
                if push_user == results[-1]['user']:
                    results[-1]['content'] += push_content
                    results[-1]['reply'] = reply

                # 若不同，表示是另一個人留的，直接新增一筆到results當中
                else:
                    results.append({"user": push_user, "type": push_type, "content": push_content, "reply": reply})
        
        # 若是該推文版主沒有回...
        else:

            # 如果目前results是空的 (初始條件)
            if len(results) == 0:
                results.append({"user": push_user, "type": push_type, "content": push_content, "reply": None})
            
            # 如果目前results不是空的，那麼就要檢查這一筆推文的作者，是不是跟results最後一筆的作者相同，如果相同的話要merge
            else:
                
                # 若該推文的作者與results最後一筆紀錄的作者相同的話，把內容合併在一起
                if push_user == results[-1]['user']:
                    results[-1]['content'] += push_content

                # 若不同，表示是另一個人留的，直接新增一筆到results當中
                else:
                    results.append({"user": push_user, "type": push_type, "content": push_content, "reply": None})
            
        
#         # 若該推文版主沒有回，且目前results是空的 (初始條件)
#         if push.endswith("</div>") and len(results) == 0:
#             results.append({"user": push_user, "type": push_type, "content": push_content, "reply": None})

#         # 若該推文版主有回，且目前results是空的 (初始條件)
#         elif not push.endswith("</div>") and len(results) == 0:
#             reply = push.split("</div>")[-1].split("<span")[0].strip()
#             results.append({"user": push_user, "type": push_type, "content": push_content, "reply": reply})

#         # 若該推文版主沒有回，且目前results不是空的
#         elif push.endswith("</div>") and len(results) != 0:

#             # 若該推文的作者與results最後一筆紀錄的作者相同的話，把內容合併在一起
#             if push_user == results[-1]['user']:
#                 results[-1]['content'] += push_content

#             # 若不同，表示是另一個人留的，直接新增一筆到results當中
#             else:
#                 results.append({"user": push_user, "type": push_type, "content": push_content, "reply": None})

#         # 若該推文版主有回，且目前results不是空的
#         elif not push.endswith("</div>") and len(results) != 0:
#             reply = push.split("</div>")[-1].split("<span")[0].strip()

#             # 若該推文的作者與results最後一筆紀錄的作者相同的話，把內容合併在一起，並記錄reply
#             if push_user == results[-1]['user']:
#                 results[-1]['content'] += push_content
#                 results[-1]['reply'] = reply

#             # 若不同，表示是另一個人留的，直接新增一筆到results當中
#             else:
#                 results.append({"user": push_user, "type": push_type, "content": push_content, "reply": reply})
    
    
    return results

## A-1: 測試

In [125]:
from pprint import pprint

results = find_list_of_comment_recomment_pairs("../data_and_code/data/marriage/2020/20200606_1746_M.1591436783.A.9E3.html")
pprint(results) 

[{'content': '相信你的直覺', 'reply': '好的，謝謝。', 'type': '推', 'user': 'w01192001'},
 {'content': '可以派徵信社的人去看嗎XD', 'reply': None, 'type': '推', 'user': 'cc000'},
 {'content': '跟蹤一次試試看，自然有答案。但沒問題就請再也不要懷疑繼續過日子',
  'reply': None,
  'type': '推',
  'user': 'pinkwl6'},
 {'content': '有辦法趁他不在時查他手機嗎？', 'reply': None, 'type': '推', 'user': 's952013'},
 {'content': '能用他手機設map的分享地點', 'reply': None, 'type': '→', 'user': 'alinglove'},
 {'content': '叫他開google地圖的軌跡給你看',
  'reply': '我試試看，這個我不太會用，研究看看，謝謝。',
  'type': '推',
  'user': 'cagari'},
 {'content': '看看行車記錄器吧',
  'reply': '有買，但是沒有安裝行車記錄器',
  'type': '推',
  'user': 'fatimalucky'},
 {'content': '問版友也沒什麼用，妳要問的是自己，如果真的抓到對方外遇，妳要怎麼做？多的是上來問怎麼辦，被戳破真相又寧願躲回洞穴內，眼不見為淨而已',
  'reply': None,
  'type': '推',
  'user': 'final007'},
 {'content': '一看就有問題啊...',
  'reply': '沒錯吧！我也覺得有問題。',
  'type': '推',
  'user': 'lebo469'},
 {'content': '70%有問題，但抓了就要有鬧翻的覺悟，不然就閉眼吧',
  'reply': '我會離婚的。',
  'type': '推',
  'user': 'pinknote'},
 {'content': '看他Google  map軌跡啊你找他有急事嗎，沒事也不用一直打，我通常是失聯那個

In [126]:
for result in results:
#     print(result)
    if result['reply'] is not None:
        print(result)

{'user': 'w01192001', 'type': '推', 'content': '相信你的直覺', 'reply': '好的，謝謝。'}
{'user': 'cagari', 'type': '推', 'content': '叫他開google地圖的軌跡給你看', 'reply': '我試試看，這個我不太會用，研究看看，謝謝。'}
{'user': 'fatimalucky', 'type': '推', 'content': '看看行車記錄器吧', 'reply': '有買，但是沒有安裝行車記錄器'}
{'user': 'lebo469', 'type': '推', 'content': '一看就有問題啊...', 'reply': '沒錯吧！我也覺得有問題。'}
{'user': 'pinknote', 'type': '推', 'content': '70%有問題，但抓了就要有鬧翻的覺悟，不然就閉眼吧', 'reply': '我會離婚的。'}
{'user': 'xxxcv', 'type': '推', 'content': '看他Google  map軌跡啊你找他有急事嗎，沒事也不用一直打，我通常是失聯那個，啊就真的在做事沒接到', 'reply': '他不是這種不接電話的人。結婚5年了，才開始這樣。'}
{'user': 'carefree1028', 'type': '→', 'content': '用衛星定位啊', 'reply': '是app嗎？有推薦的嗎？'}
{'user': 'trueblend', 'type': '推', 'content': '好奇請問妳們最後把洗衣機放哪一樓?有加買烘衣機嗎？', 'reply': '哈，最後買2台洗衣機，一台2樓後面，一台4樓，主要曬衣是4樓。'}
{'user': 'novagirl', 'type': '→', 'content': '2-3小時算失聯...你每小時打給他嗎?', 'reply': '他是一個手機幾乎不離身的人，最近這樣，我是每小時打一次沒錯。而且剛好2-3小時，\n就會回撥了，好巧呢！'}
{'user': 'djboy', 'type': '推', 'content': '如果最近才這樣，有二個可能：(1)無性關係：就是他對你或是需要自己一個人舒壓。你可以想想自己最近和

## A-2: 對目錄中的每個檔案都做抓取pair

In [14]:
m = '/Users/honghantang/corpus_ptt/some_ptt_posts_from_132/marriage_2011/'
g = '/Users/honghantang/corpus_ptt/some_ptt_posts_from_132/gay_2010/'

def get_comment_recomment_pairs_in_folder(folder_path):
    
    import glob

    post_comment = dict()

    for filename in glob.glob(folder_path + '*.html'):

        dict_key = filename.lstrip(folder_path).rstrip(".html")
        rs = find_list_of_comment_recomment_pairs(filename)      
        post_comment[dict_key] = rs
    
    return post_comment    

In [24]:
mm = get_comment_recomment_pairs_in_folder(m)
gg = get_comment_recomment_pairs_in_folder(g)

In [19]:
for post_id in gg:
    for reply in gg[post_id]:
        if reply['reply'] is not None:
            print(reply)

{'user': 'hkyuppy', 'type': '→', 'content': '上星期,蘋果日報.有報導:在警光被人摸, 有去的朋友.要小心要確定對方是,才可以哦,祝大家新年快樂,有"快樂"溫泉行http://ppt.cc/7e4I這是相關的報導,請大家留心囉.才能大家都快樂啦', 'reply': '↑ 幫你縮網址'}
{'user': 'kuanchang', 'type': '→', 'content': 'g大有哪個字說他認同遲到了?而且慫恿這個字你也講得出來?', 'reply': '為什麼講不出來？\n妳自己才是睜眼說瞎話。'}
{'user': 'b822187', 'type': '噓', 'content': '發心情文是很多沒錯 但你沒必要這樣洗版吧 可以大E修文回', 'reply': '======================================================\n\n好好笑。\n\n我連一頁都還沒洗過去，不過發表了幾篇文，洗了什麼版？\n洗版是個很嚴重的指控，代表文章過多、灌水、且嚴重影響大家的閱讀。\n請問我哪點符合洗版了？\n妳看不爽我的文，就不要看不要回。\n不要給老娘亂扣帽子。'}
{'user': 'mrdragon', 'type': '→', 'content': '所以是我一開始的提問太過壓迫?原來如此 所以是我說的方法錯誤這樣回頭來看 我的問法的確會給人壓力感回s大： 我只是認為這樣可以讓人去認識、了解事情的本質感謝你很有耐心的說明', 'reply': '剛剛斷線  不管你是要拉什麼樣的污名族群去遊行\n公開現身 參加活動\n最基本的動作是  你應該讓對方知道你是支持他的\n到了那個現場你會罩他\n\n如果你前面的脈絡讓人感受不到支持\n那這樣的提問看不出來有什麼平權的鼓勵性質 反而是一種壓迫\n\n而且你一直把支持用藥跟自己用藥畫上等號\n如果事情要這樣搞 把支持者跟當事人混為一談\n可能連同志遊行都不會成長 最早在同志遊行人還很少\n不是像現在可以藏在一堆人裡面不太會被注意到的時候\n是靠許多異性戀或友善團體的參與才有的\n\n至於需要認識 了解事情\n前面已經說過 同志諮詢熱線有辦講座\n那個溝通會比較有深度 遊行反而有它的挶限'}
{'user': 'benka', 't

In [22]:
def count_recomment_percentage(dict_of_pairs):
    
    total_post = len(dict_of_pairs)
    
    
    total_post_without_any_recomment = 0
    
    total_comment = 0
    total_replied_comment = 0

    for post_id, list_of_reply in dict_of_pairs.items():
        
        total_comment += len(list_of_reply)
        total_replied_comment += len([elm for elm in list_of_reply if elm['reply'] is not None])
        
        without_any_reply = True
        
        for reply in list_of_reply:
            if reply['reply'] is not None:
                without_any_reply = False
                break
                
        if without_any_reply:
            total_post_without_any_recomment += 1
    
    print(f"總文章數: {total_post}")
    print(f"無版回復的文章數: {total_post_without_any_recomment}")
    print(f"版主回覆數 / 總推文數 = {total_replied_comment/total_comment}%")

In [23]:
count_recomment_percentage(gg)

總文章數: 428
無版回復的文章數: 384
版主回覆數 / 總推文數 = 0.011735330836454432%


In [8]:
for post_id in post_comment:
    print(f"[[ {post_id} ]]")
    for reply in post_comment[post_id]:
        if reply['reply'] is not None:
            print(reply)

[[ 945_M.1296524710.A.813 ]]
[[ 824_1743_M.1314179002.A.BBF ]]
[[ 9_0932_M.1297215133.A.F4E ]]
[[ 919_2312_M.1316445125.A.CDD ]]
[[ 427_1940_M.1303904436.A.F74 ]]
[[ 823_0002_M.1314028939.A.EC1 ]]
[[ 609_0836_M.1307579789.A.7E3 ]]
[[ 5_1650_M.1299315016.A.9F3 ]]
[[ 504_1714_M.1304500451.A.2A7 ]]
[[ 823_1657_M.1314089860.A.A42 ]]
[[ 711_2346_M.1310399218.A.A41 ]]
[[ 49_M.1320158977.A.907 ]]
[[ 9_1031_M.1325125867.A.E39 ]]
[[ 815_1826_M.1313404014.A.628 ]]
[[ 408_2334_M.1302276890.A.102 ]]
[[ 928_2349_M.1317224983.A.591 ]]
[[ 518_1932_M.1305718323.A.2F2 ]]
[[ 614_1544_M.1308037496.A.BFB ]]
[[ 519_1602_M.1305792124.A.19D ]]
[[ 6_2311_M.1324048315.A.E7D ]]
[[ 6_1449_M.1301122176.A.B79 ]]
{'user': 'youame', 'type': '→', 'content': '妳想搬出去就自己來比較快,到時房子買好,妳和小孩有地方住你老公看他要住他家或妳那就隨便他,至少妳小孩在妳身邊妳真的要自己硬起來,都成年人了,不用什麼都要老公去處理妳都已經心裡壓力大到想在高速公路跳車了,再下去心裡會生病反正都做好離婚的打算了,那還有什麼好怕的,就去做吧..', 'reply': '我把那段刪了，我也知道那很不理智，可是明明說好的事，卻不遵守，\n我說我要靠自己買房子，他也說他絕不會跟我住，而且小孩也絕對不\n會讓我帶走．．．．小孩是我的命，沒有他，我真的活不下去\n\n他以前明明說為了存錢，所以先不在蘆洲

[[ 426_1902_M.1303815768.A.A53 ]]
[[ 4_M.1318520065.A.72D ]]
[[ 4_2124_M.1295875488.A.3C6 ]]
[[ 7_0300_M.1319655620.A.463 ]]
{'user': 'urmomo', 'type': '噓', 'content': '不知道你第三篇的重點是在畫什麼人家當soho錢賺的比以前多 工時比以前少也不行喔人家有本事日子過太爽關你啥事 原文中的老婆要是自己做soho拿錢出來請人來雇小孩或打掃 老公會反對嗎我想起來了 顧小孩會忙到連杯子都沒時間洗 哪有可能作soho第二篇我也不知道你的重點在哪裡 是在批原文的太太沒顧好小孩嗎 看看人家當女兒的受寵成這樣 當原文中的小孩只能手指藏污納垢 是這個意思嗎', 'reply': '第二篇重點在  如果自家媽媽知道女兒照顧兩個小孩加一條大狗  結果丈夫因為\n那些原因怪她不找工作又沒把家顧好  不知道會說啥:)\n\n端看妳怎想拉  反正別人家女兒都該好  自家女兒寵當寶'}
[[ 716_M.1320138983.A.EF5 ]]
{'user': 'amberyen', 'type': '→', 'content': '婆媳問題有些時候很難說誰是誰非。但看你這樣嘲笑不懂的用柺杖的長輩，還蠻令人心寒的', 'reply': '是因為不是真的需要用到拐杖~硬要裝出來的'}
{'user': 'penlyly', 'type': '→', 'content': '柺杖不一定是夾腋下喔 要看使用者需求', 'reply': '其實她更希望有人推輪椅~之前姑姑推輪椅帶他出門.在公園反而因為卡到跌下來\n不敢再坐輪椅.於是她就可以自己走路了= ='}
{'user': 'longloveyu', 'type': '推', 'content': '妳舉的這三點 跟婆媳問題有什麼關係?第一點,阿嬤是邊罵令堂髒邊說自己愛乾淨?第2點,阿嬤說令堂不會做菜,順便 說自己都做紅龜糕?第3點,令堂把妳阿嬤弄跌倒?', 'reply': '跌倒不是媽媽弄得~是奶奶自己跌的'}
{'user': 'bunnys', 'type': '噓', 'content': '洗文章太辛苦了啦,還是回去幫阿嬤洗冰箱吧', 'reply': '不好意思我家冰箱

[[ 606_1817_M.1307355455.A.AEA ]]
[[ 728_0859_M.1311814757.A.282 ]]
[[ 624_1705_M.1308906309.A.956 ]]
[[ 617_1422_M.1308291769.A.3E6 ]]
[[ 920_1733_M.1316511211.A.6F7 ]]
{'user': 'ohmytrp', 'type': '噓', 'content': '我覺得妳好奇怪= = 別人洗碗也不行？', 'reply': '你這樣亂噓人比較奇怪吧\n我有說誰洗碗不行嗎?\n我只是覺得背後小人的行為很討厭'}
[[ 414_1304_M.1302757497.A.D73 ]]
{'user': 'kusakapt', 'type': '推', 'content': '推一樓，搞得好像人可以無性生殖。', 'reply': '給具體作法比歸咎責任好些。'}
[[ 6_1120_M.1297826430.A.B21 ]]
[[ 721_M.1296516068.A.5D8 ]]
[[ 615_1949_M.1308138586.A.9CB ]]
[[ 807_1107_M.1312686451.A.AAD ]]
[[ 6_2217_M.1318774655.A.DD7 ]]
{'user': 'condenza', 'type': '推', 'content': '對耶 還有哥哥 也不必自己提', 'reply': '忍不住想解釋一下~其實是工廠的歐巴桑~~\n\n       當時大家在聊天~所以提前剖腹跟我上一胎是男的等話題才剛聊完~\n\n       且我兩胎都在這間公司任職時生的~所以~好事的歐巴桑當然也對這些很清楚囉~'}
{'user': 'gogoms', 'type': '推', 'content': '這篇的作者可以去看上一篇的媽媽,她真的很有智慧didadi大寫的...我也分享自己的經驗吧,我跟我弟弟的女友很要好,我很喜歡她,我懷孕時她跟我說,妳預產期是天秤座喔?然後跟我的肚子說:你晚點出來啦,天蠍男比較酷!!當時我一直大笑,覺得她好三八,沒有想很多...大概是因為我喜歡她所以覺得她什麼都是為我好,人就是這樣,如果是討人厭的原原po大嫂或是這位歐巴桑講出天蠍座的話題,我想妳們還是會氣爆

[[ 525_1237_M.1306298260.A.0F0 ]]
[[ 824_M.1300875895.A.61C ]]
[[ 519_1346_M.1305784001.A.845 ]]
[[ 421_2006_M.1303387593.A.93F ]]
[[ 8_1417_M.1320733073.A.9E1 ]]
[[ 707_1002_M.1310004137.A.59F ]]
{'user': 'eva19452002', 'type': '→', 'content': '這不叫多愁善感，是妳婆婆太過份了，我就不信相同的話她會對她嫁出去的女兒說自己的身體最重要，就算黑掉，也要堅持坐滿月子', 'reply': '回Kill大..上次一要搬，我就出血了（五月時）XD，，我娘家媽媽嚇得半死，\n然後新房沒冷氣最近才裝好. 所以拖到這禮拜搬\n另外回akj大..我沒在婆家坐月子..老公是很無奈..因為他也很囧.\n他一直跟我說不要理他媽，因為公婆會在我不在時一直逼問他我的事\n他怎麼說，公婆都會跳針\n至於我怎麼這麼堅持找工作，可能一部份我不想被看輕吧，\n我那時安胎辭工作，我公婆就很有意見，我老公幫我講話公婆也聽不進去，\n只說：只會幫老婆說話'}
[[ 805_0340_M.1312486821.A.876 ]]
[[ 5_0001_M.1296835286.A.9EE ]]
{'user': 's866217', 'type': '噓', 'content': '不會推文呀', 'reply': '推文不是用來討論東西用的，推文是用來起鬨鬧事用的。'}
{'user': 'jayjay6125', 'type': '→', 'content': '原PO把自己得感受說出來也要被酸...真衰', 'reply': '要一個對著不愛自己的男人幻想他愛自己的女人承認真相，\n比駱駝穿過針眼更難。\n\n她有權發表自己的感想，我們有權發表評論。\n\n只給拍拍的笨女人們是推這個笨女人下地獄的幫兇，\n我們不幹這種事。'}
{'user': 's866217', 'type': '噓', 'content': 'loser1:是起鬨啊..... 自己都說是起鬨了 你還不推文 打臉→ loser1:是起鬨啊.....自己說自己是起鬨的', 'reply': '這篇

[[ 6_1311_M.1321420316.A.774 ]]
[[ 5_1348_M.1321336121.A.547 ]]
[[ 603_1926_M.1307100389.A.778 ]]
[[ 906_1533_M.1315294417.A.35E ]]
[[ 711_1106_M.1310353583.A.3EA ]]
[[ 9_0245_M.1321641933.A.3AE ]]
{'user': 'peiy', 'type': '推', 'content': '原po被鞭的理由應該不是眾人認為不能認賠殺出 而是不能把孩子丟在她不想待的爛環境中然後自己逃難去追求自己的幸福人生你文中也說了為在這樣環境中的孩子們感到難過', 'reply': '我PO這篇文，\n只是想對&#34;先生還沒到酗酒的程度，情況沒有那麼嚴重&#34;這個想法\n表達我的看法，也說一下自己看到的事情而已\n並不是想對原原PO為什麼被鞭來加以解釋或是護航\n\n我也不認同原原PO對孩子做的決定\n所以文章後面有提到幾點，是我認為原原PO如果真的想把孩子交給夫家\n所要認真去考慮的情形及後果\n畢竟孩子的價值觀如何被塑造，所影響的是孩子的整個人生\n甚至是與這個孩子有所交集的人的人生，都在影響範圍之內!\n\n以前曾經看過一本書，書名是《再見，寶貝，再見》\n裡面有一段話，是身為偵探的主角對那些虐童犯的描述\n內容大約是這樣: 那些人(文中指虐童犯的父母)在孕育這些生命時，\n                知不知道他們將孕育出的是一個怪物?\n\n這是一段很驚悚的說詞，可是卻很明白的說出一個人的教育養成有多重要\n所以原原po在放下孩子之前，想的不應該是自己的、原生家庭的或是孩子的物質\n而是那個教養孩子環境，會不會讓你以後對現在下的決定後悔莫及'}
[[ 7_2128_M.1297949298.A.B7F ]]
[[ 528_1011_M.1306548714.A.C8B ]]
[[ 607_1609_M.1307434186.A.06A ]]
{'user': 'nanahiei', 'type': '推', 'content': '不用太認真啦,先不提身體的問題(如果知道還問就白目了)年紀到了就會被問何時嫁娶,成家後就問何時生子,生一個後問何時再添個伴,有了二個以上就問小孩讀哪

[[ 5_2151_M.1300197102.A.0DA ]]
{'user': 'gypiq', 'type': '推', 'content': '請問什麼樣的男人值得愛?什麼權的男人值得踹?', 'reply': '壞人就該踹。\n\n愛上不值得愛的男人，他不娶你回家不是他的錯，他娶你回家就是他的錯，\n該踹。\n\n吃老婆用老婆還跑去打電動不念書的男人，該踹(請參閱五星物語第十集，\n女人專用招數)。'}
[[ 8_1243_M.1300423390.A.F56 ]]
[[ 601_1626_M.1306916768.A.E83 ]]
[[ 4_1837_M.1298543876.A.870 ]]
[[ 906_1906_M.1315307202.A.FF5 ]]
[[ 426_1312_M.1303794776.A.2F4 ]]
[[ 828_1158_M.1314503893.A.5CB ]]
{'user': 'jlo67', 'type': '→', 'content': 'i媽 這裡的風氣可沒媽寶版那麼溫馨、平和 什麼人都有 有些發言要適當忽略 也要留意別太入別人家的戲 XD', 'reply': 'eva19452002大！我不管你是男是女,是認真還是胡鬧,看過很多板都被\n一些太無聊的人胡搞,若您覺得太空太無聊,請出去走走,不要回文!!'}
[[ 9_0130_M.1297186212.A.B8F ]]
[[ 702_2204_M.1309615490.A.0F8 ]]
[[ 412_1134_M.1302579261.A.2E1 ]]
{'user': 'melodypiano', 'type': '推', 'content': '父親的角色消失了 囧', 'reply': '*→ *blockcat*[m*:真的耶，因為爸爸的實感好像是跟小孩相處下來累積?\n*→ *blockcat*[m*:果分開住都沒當爸的實感了，真的後來會疼小孩嗎？\n*→ *blockcat*[m*:這一代的爸媽真可憐(？) 大概是上一代不能理解的吧\n\n\n          是指我老公嗎XD\n\n          他工作比較忙 也不一定休六日\n          但他休假一定會跟著我回去陪小孩\n          (所以還好我一週內還是有幾天不必單獨面對

{'user': 'chaiho', 'type': '推', 'content': '我娘家吃飯都自己盛，自己裝自己的不是很好嗎??', 'reply': '是啊，我娘家吃飯，也是自己盛自己的，但是我們（還沒結婚的時候）會幫老爸盛飯。\n\n老公家很&#34;日式&#34;（我婆婆自己常批評公公：哼，日本好的優點沒學到，學來一堆爛規矩）\n菜擺好，盤子（放菜渣 骨頭的）擺好，筷子擺好，飯盛好，然後叫公公及大家來吃飯。\n\n我婆婆生了四個兒子，沒有女兒，所以他們從小在這種『遠庖廚』的環境下長大，說實在\n這種完全不幫忙、等著&#34;飯來張口&#34;的教育方式非常的失敗，我個人非常不齒，所以我的孩\n子慢慢長大，我會要求他們幫忙拿筷子、盤子，拿小凳子踩高、學著自己盛飯（就算現在\n有請外傭也是如此）… 然後，我婆婆就開始跟小孩說啦：\n\n「你老母這麼柄哆（懶惰），叫你們去盛飯喔?!」\n「你老母只會叫你們做這做那…」\n小孩還小，不知道阿嬤講的台語是什麼意思，我婆婆樂得放聲批評…\n\n我寫這個往事的重點是，就算隨著時間的演進，不管過了幾年，若是我婆婆幫大家盛飯，\n有大伯的、有小叔的、也有小嬸（小叔的老婆）的、有孫女的…也有她自己的，就一定沒\n有我的，這個現象很有趣，她自己不解釋，妳也搞不清楚她是哪種堅持。\n\nC2C大大不必急著演苦情劇啦，我婆婆以前是留美的，曾任台大護理長之類的（管護士們）\n不是你想的那樣啦XD\n\n我老公說：「自古以來婆媳問題從來沒有中斷過，要能住在一起，必須要有足夠的智慧，\n我以前一直以為我媽有這種智慧…」\n\n不知道為什麼，我還滿享受我婆婆把我當外人的這種待遇，因為我一直都認為，我本來就\n是我爸媽的女兒並引以為榮，我本來就不是他們林家的人啊！哈哈哈'}
{'user': 'leprosy', 'type': '→', 'content': '台大護理長才傳統啊,樓上學妹實在沒深入的了解', 'reply': '真想給 drama 100個讚，您點出了我的本意。\n\n我猜啊，大家都還很年輕（真好），所以看一個故事總會希望『結果呢？』『結局呢？』\n『沒什麼改變嘛！』……可是真實的人生不是這樣演的。\n\n先想想自己，通常是我們自己有所體悟，才會調整我們自己的心態和做法。但是如果&#34;妄\n想&#34;改變他人，那可

{'user': 'Avril1975', 'type': '→', 'content': '看到這個話題想去改成小孩@@', 'reply': '其實我想變更成理賠金的佔比應該也可以吧?\n不過希望永遠都不要動用到!'}
{'user': 'plarse', 'type': '→', 'content': '不過你老公為何會強調是為你繳的 還是你誤會他意思', 'reply': '每次保險繳費單一來，他就說:又要繳妳的保單了 :&lt;'}
{'user': 'Nicowyaw', 'type': '→', 'content': '買張新保單，佔比隨你安排', 'reply': '我想這應該是最好的方式'}
{'user': 'weilin07', 'type': '→', 'content': '妳 所以改不改應該沒差吧', 'reply': '恩 這樣也沒錯'}
{'user': 'Nicowyaw', 'type': '推', 'content': '你可以轉念很好，可是他說那句[又要繳你的保單]實在是><', 'reply': '因為他在高風險的場所上班，他覺得幫他自己買份保險是給我的保障\n所以他才會說又要繳妳的保單\n受益人會寫公公是因為當初我們還沒結婚\n\n不過說真的他有這份心我很高興\n況且看版友討論，後來我覺得就算受益人一直不變更我也不在意了\n畢竟父母養育之恩比天高\n\n自立自強重新買份保單比較實在!'}
{'user': 'ladyluck', 'type': '推', 'content': '還是覺得要給老婆保障為何不是婚後買....', 'reply': '這個我也不知道，就要問他本人了 :&lt;'}
{'user': 'plarse', 'type': '→', 'content': '受益人並不需要你是老婆才能填呀 跟婚前婚後有什麼關係XD還是覺得妳老公說那是給你的保障 然後因為當時是"女友"所以不填你當受益人 很弔詭不過你轉念了 填誰都沒差的話 就不用追究了XDD', 'reply': '是的'}
[[ M.1322835645.A.EAC ]]
[[ 5_2352_M.1298649131.A.908 ]]
[[ 609_2044_M.1307623458.A.193 ]]
[[ 9_1407_M.1323410862.A.

[[ 4_2219_M.1322144357.A.F5A ]]
[[ 8_0829_M.1318897748.A.CC7 ]]
[[ 7_1634_M.1297931665.A.593 ]]
[[ 519_1738_M.1305797899.A.92B ]]
[[ 8_1722_M.1321608132.A.AE2 ]]
[[ 815_1312_M.1313385141.A.B3B ]]
[[ 906_1352_M.1315288325.A.AB7 ]]
{'user': 'amberyen', 'type': '推', 'content': '媽媽寵、姊妹寵、老婆寵....', 'reply': '這句話說的好！哈哈哈，所以我常稱呼他少爺，少爺都是我們自己養出來的\n我沒有阻止他去，因為雖然燒到39度多很難過，但也不過就是感冒\n\n套我小姑的說法，她哥就是「靠勢」家裡人力充足，願意幫忙，才會這樣\n善於運用調配資源沒什麼不對啦，就是情感上覺得真有點機車這樣\n看著她哥，小姑說等她自己成家立業，一定要兩人小家庭，不讓先生有機可趁，哈！'}
{'user': 'prissy', 'type': '→', 'content': '感謝的話不用告訴我們,應該直接說給小姑聽吧!', 'reply': '我講啦！這個婚姻帶給我好幾位新的好家人，真的很感謝~~~\n謝謝大家的關心，退了燒就好多了，所以才能來po文，還下標題是「搞笑」\n雖然還是覺得有點<a href="/cdn-cgi/l/email-protection" class="__cf_email__" data-cfemail="032243">[email&#160;protected]</a>#$，但我是能接受啦，因為不在情但還是算蠻在理的(?)....\n只是他這次發包交辦的理所當然還面面俱到，實在讓人又好氣又好笑，\n就分享一下，這世界上還是有這種夫妻的，這樣還是可以平和度日的啦\n\n喔對，今天晚上他還是弄了一堆海鮮給我當晚餐，有網友推文說不合適喔？\n算了，他的好意嘛我還是吃了，還很開心...我愛海鮮！不知道等等會不會怎樣？再說吧！\n還是謝謝大家的關心！'}
[[ 702_1732_M.1309599177.A.5C9 ]]
[[ 5_0114_M.1321290846.A.95F ]]
[[ 6

# B. 抓出標題含有 Re:

In [25]:
def all_posts_in_path_generator(path):
    for filename in glob.glob(f"{path}/*.html"):
        with open(filename, "r") as f:
            ff = f.read()
        yield BeautifulSoup(ff)


In [27]:
def has_re_in_title(bs):
    return bs.title.text.startswith("Re: ")

In [30]:
for x in all_posts_in_path_generator(g):
    if has_re_in_title(x):
        print(x.title.text)

Re: [家庭] 關於父親 - 看板 gay - 批踢踢實業坊
Re: [心情] 紀念玫瑰少年葉永鋕 莫忘尊重不同性別特質 - 看板 gay - 批踢踢實業坊
Re: [新聞] 帥警扮買家 破獲禁藥 - 看板 gay - 批踢踢實業坊
Re: [心情] 讀者來函 - 看板 gay - 批踢踢實業坊
Re: [轉貼] 所謂拉K成癮 - 看板 gay - 批踢踢實業坊
Re: [閒聊]甲板都不甲板了 - 看板 gay - 批踢踢實業坊
Re: [心情] 關於老同志 - 看板 gay - 批踢踢實業坊
Re: [家庭] 關於父親 - 看板 gay - 批踢踢實業坊
Re: [請益] 版主是不是能對於pureman88的處分作出묠… - 看板 gay - 批踢踢實業坊
Re: [新聞] 罵GAY不起訴 罵死GAY罰6000 - 看板 gay - 批踢踢實業坊
Re: [心情] 讀者來函 - 看板 gay - 批踢踢實業坊
Re: [請益] 第一次聽說有甲湯 - 看板 gay - 批踢踢實業坊
Re: [請益] 有人可以幫忙翻譯這一段影片嗎...為巴ꐠ… - 看板 gay - 批踢踢實業坊
Re: [請益] 第一次聽說有甲湯 - 看板 gay - 批踢踢實業坊
Re: [心情] 稍微找了一下關於藥的資訊 - 看板 gay - 批踢踢實業坊
Re: [心情] 禁色：我和異男的飲茶時光 - 看板 gay - 批踢踢實業坊
Re: [請益] 關於10這件事 - 看板 gay - 批踢踢實業坊
Re: [討論] 娘的市場大不大？ - 看板 gay - 批踢踢實業坊
Re: [怨念] 莫名其妙的水球 - 看板 gay - 批踢踢實業坊
Re: [公告] gay 賭盤開獎 - 看板 gay - 批踢踢實業坊
Re: [分享] 色淫師之台北新莊Motel拍拍走‧Bryan - 看板 gay - 批踢踢實業坊
Re: [版務] gay板 kaorikuraki 板主請假 - 看板 gay - 批踢踢實業坊
Re: [版務] gay板 kaorikuraki 板主請假 - 看板 gay - 批踢踢實業坊
Re: [新聞] 驗愛滋篩出同志　積分飆高 - 看板 gay - 批踢踢實業坊
Re: [心情] 為巴比祈禱之後的難熬 - 看板 gay - 批踢踢實業坊
Re: [心情] 讀者來函 - 看板

# C. 處理Repo中的原文與新文

In [22]:
with open("../data_and_code/data/marriage/2020/Re [求助] 家裡事業與小孩陪伴問題.html") as f:
    h = f.read()
bs = BeautifulSoup(h)

In [63]:
import bs4

def parse_new_and_old_information_in_repo(soup):
    results = list()
    x = soup.find("div", id="main-content")
    for line in x:
    #     print(line)

        # 代表是新的文章
        if isinstance(line, bs4.element.NavigableString):
            try:
                if len(line.strip()) == 0:
                    continue

                if results[-1]["new"]:
                    results[-1]["content"].append(line)
                else:
                    results.append({
                        "new": True,
                        "content": [line]
                    })
            except IndexError:
                results.append({
                        "new": True,
                        "content": [line]
                    })

        # 只有 <class="f6"> 的才是原文
        elif line.name == "span" and "f6" in line['class']:
            try:
                if not results[-1]["new"]:
                    results[-1]["content"].append(line.text)
                else:
                    results.append({
                        "new": False,
                        "content": [line.text]
                    })
            except IndexError:
                results.append({
                        "new": False,
                        "content": [line.text]
                    })

        else:
            continue
            
    return results

In [62]:
r = []
for x in all_posts_in_path("../data_and_code/data/marriage/2020"):
    if having_re_in_title(x):
        r.append(parse_new_and_old_information_in_repo(x))

<span class="f6">: 開始會買名牌包從3萬的開始買一直買到12萬一個包
</span>
<span class="f6">: 小弟從小就被教導要省水省電、物盡其用
</span>
<span class="f6">: 老公跟我說，
</span>
<span class="f6">: 小弟從小就被教導要省水省電、物盡其用
</span>
<span class="f6">: 身為一個傳產窮二代，小弟也來小小回文一下
</span>
<span class="f6">: 小弟從小就被教導要省水省電、物盡其用
</span>
<span class="f6">: 小弟從小就被教導要省水省電、物盡其用
</span>
<span class="f6">: 小弟從小就被教導要省水省電、物盡其用
</span>
<span class="f6">: 小弟從小就被教導要省水省電、物盡其用
</span>
<span class="f6">: 文章有點長算是心情抒發…
</span>
<span class="f6">: 是我太津津計較嗎？
</span>
<span class="f6">: 小弟從小就被教導要省水省電、物盡其用
</span>
<span class="f6">: 小弟從小就被教導要省水省電、物盡其用
</span>
<span class="f6">: 我是原PO
</span>
<span class="f6">: 小弟從小就被教導要省水省電、物盡其用
</span>


In [70]:
for r_ in r:
    print(len(r_))

1
1
1
2
14
2
1
29
16
1
2
14
1
7
1
26
3
1
18
2
17
17
3
1
3
16
1
2
24
15
1
3
17
33
1
16
1
3
1
2
1
1
3
15
3
37
14
37
1
